# Disaggregation with PyTorch and lstms

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

from torch.nn import MSELoss
from torch.optim import Adam

from distorch.dataset import DisaggregationDataset
from distorch.trainer import Trainer
from distorch.metrics import PerformanceEvaluator

from distorch.models.lstm_disaggregator import *

In [ ]:
dd = DisaggregationDataset(preprocess=True, months=[2, 8])
train_set, val_set = dd.get_train_val_set()
train_loader, val_loader = dd.get_dataloader(batch_size=64)

In [ ]:
disaggregator = LSTMDisaggregator(
    n_features=9, hidden_dim=9, n_outputs=2
)

trainer = Trainer(
    disaggregator, MSELoss(reduction='mean'), Adam(disaggregator.parameters(), lr=0.001)
)

trainer.set_loaders(train_loader, val_loader)
trainer.train(50)

In [ ]:
X_val, y_val = val_set.tensors

pv_pr, load_pr = trainer.predict(X_val)
pv_true = y_val[:,1]
load_true = y_val[:,0]

In [ ]:
pv_eval = PerformanceEvaluator(pv_pr, pv_true, scale=1.41)
load_eval = PerformanceEvaluator(load_pr, load_true, scale=5.18)

print(pv_eval.get_performance_metrics())
print(load_eval.get_performance_metrics())

In [ ]:
load = pd.DataFrame(
    index=dd.val_index
)
load['load_pred'] = load_pr
load['load_true'] = load_true

load[(load.index.month == 9) & (load.index.day.isin([4,5,6]))].plot();

In [ ]:
pv = pd.DataFrame(
    index=dd.val_index
)
pv['pv_pred'] = pv_pr
pv['pv_true'] = pv_true

pv[(pv.index.month == 10) & (pv.index.day.isin([4,5,6]))].plot();

In [ ]:
from distorch.models.mlp_disaggregator import MLPDisaggregator

disaggregator = MLPDisaggregator(
    hidden_layers = 4 * [9]
)

trainer = Trainer(
    disaggregator, MSELoss(reduction='mean'), Adam(disaggregator.parameters(), lr=0.001)
)
trainer.set_loaders(train_loader, val_loader)
trainer.train(1000)

In [ ]:
X_val, y_val = val_set.tensors

pv_pr, load_pr = trainer.predict(X_val)
pv_true = y_val[:,1]
load_true = y_val[:,0]

pv_eval = PerformanceEvaluator(pv_pr, pv_true, scale=1.41)
load_eval = PerformanceEvaluator(load_pr, load_true, scale=5.18)

print(pv_eval.get_performance_metrics())
print(load_eval.get_performance_metrics())

In [ ]:
load = pd.DataFrame(
    index=dd.val_index
)
load['load_pred'] = load_pr
load['load_true'] = load_true

load[(load.index.month == 6) & (load.index.day.isin([4,5,6]))].plot();

In [ ]:
pv = pd.DataFrame(
    index=dd.val_index
)
pv['pv_pred'] = pv_pr
pv['pv_true'] = pv_true

pv[(pv.index.month == 10) & (pv.index.day.isin([4,5,6]))].plot();